In [ ]:
import pandas as pd
import numpy as np
import nltk
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, classification_report
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from nltk.tokenize import word_tokenize

In [ ]:
import csv

texts = []
labels_ = []

# Open the CSV file and read it line by line
with open("/content/complaints.csv", "r", encoding="utf-8") as file:
    # Create a CSV reader
    csv_reader = csv.reader(file)
    # Skip the header line
    next(csv_reader)
    # Iterate over each row in the CSV file
    for row in csv_reader:
      # Extract text and label from the row
      text = row[0]  # Remove leading and trailing quotes
      if len(row) > 1:
        label = row[1]  # Extract label from the second column
      else:
        label = None
      # Append text and label to the respective lists
      texts.append(text)
      labels_.append(label)

In [ ]:
nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:

# Preprocess the text data
texts = [word_tokenize(text.lower()) for text in texts]

# Encode the labels
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels_)


In [ ]:
# Convert texts to sequences and pad them
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
max_sequence_length = max(len(seq) for seq in sequences)
data = pad_sequences(sequences, maxlen=max_sequence_length)

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

In [ ]:
# Define model parameters
embedding_dim = 100
vocab_size = len(tokenizer.word_index) + 1
num_classes = len(label_encoder.classes_)

# Define the CNN model architecture
model = tf.keras.models.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_sequence_length),
    tf.keras.layers.Conv1D(128, 5, activation='relu'),
    tf.keras.layers.GlobalMaxPooling1D(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

# Compile the model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=10, batch_size=64, validation_split=0.2)

Epoch 1/10
90/90 [==============================] - 23s 207ms/step - loss: 0.9602 - accuracy: 0.6894 - val_loss: 0.7687 - val_accuracy: 0.7088
Epoch 2/10
90/90 [==============================] - 17s 193ms/step - loss: 0.5978 - accuracy: 0.7783 - val_loss: 0.5449 - val_accuracy: 0.7926
Epoch 3/10
90/90 [==============================] - 14s 157ms/step - loss: 0.3815 - accuracy: 0.8705 - val_loss: 0.4504 - val_accuracy: 0.8373
Epoch 4/10
90/90 [==============================] - 14s 157ms/step - loss: 0.2220 - accuracy: 0.9312 - val_loss: 0.4444 - val_accuracy: 0.8506
Epoch 5/10
90/90 [==============================] - 14s 162ms/step - loss: 0.1129 - accuracy: 0.9701 - val_loss: 0.4805 - val_accuracy: 0.8422
Epoch 6/10
90/90 [==============================] - 15s 161ms/step - loss: 0.0463 - accuracy: 0.9925 - val_loss: 0.5345 - val_accuracy: 0.8485
Epoch 7/10
90/90 [==============================] - 11s 120ms/step - loss: 0.0233 - accuracy: 0.9967 - val_loss: 0.5559 - val_accuracy: 0.8436

In [ ]:
# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')

56/56 [==============================] - 1s 12ms/step - loss: 0.5627 - accuracy: 0.8520
Test Loss: 0.5627123713493347, Test Accuracy: 0.8520379662513733


In [ ]:
# Predict the probabilities for the test data
y_pred_prob = model.predict(X_test)

# Convert predicted probabilities to class labels
y_pred = np.argmax(y_pred_prob, axis=1)

# Convert encoded labels back to original labels
y_test_original = label_encoder.inverse_transform(y_test)
y_pred_original = label_encoder.inverse_transform(y_pred)

# Generate classification report
report = classification_report(y_test_original, y_pred_original)
print(report)

56/56 [==============================] - 0s 7ms/step
                                      precision    recall  f1-score   support

   Attempts to collect debt not owed       0.68      0.73      0.71       379
               Communication tactics       0.60      0.59      0.60        61
                       Fraud or scam       0.73      0.86      0.79        43
Incorrect information on your report       0.93      0.90      0.91      1230
          Struggling to pay mortgage       0.86      0.83      0.84        78

                            accuracy                           0.85      1791
                           macro avg       0.76      0.78      0.77      1791
                        weighted avg       0.86      0.85      0.85      1791



In [ ]:
import matplotlib.pyplot as plt
conv_layers = [layer for layer in model.layers if isinstance(layer, tf.keras.layers.Conv1D)]

# Visualize filters for each convolutional layer
for i, layer in enumerate(conv_layers):
    filters = layer.get_weights()[0]
    num_filters = filters.shape[2]
    
    # Plot each filter as a grid
    plt.figure(figsize=(10, 10))
    for j in range(num_filters):
        plt.subplot(num_filters/8, 8, j+1)
        plt.plot(filters[:, :, j])
        plt.axis('off')
    plt.suptitle(f'Conv1D Layer {i+1}')
    plt.show()